In [303]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import random
import re
import json
import pandas as pd
import numpy as np

In [304]:
url = 'https://www.formbot3d.com/products/voron-24-corexy-3d-printer-kit-with-different-print-sizes-for-choice'

In [305]:
# chrome_options = Options() 
# chrome_options.add_argument('--no-sandbox') 
# chrome_options.add_argument('--disable-dev-shm-usage')

In [306]:
# automatically use the correct chromedriver by using the webdrive-manager
path = ChromeDriverManager().install()

In [307]:
# initialize and close the driver (new): 
driver = webdriver.Chrome(path)

# initialize and close the driver (old): error SessionNotCreatedException
# driver = webdriver.Chrome(path)

# make sure that the page has time to properly open 
#time.sleep(3) 

driver.get(url)

assert 'Voron 2.4 CoreXY 3D Printer Kit with Different Print Sizes for Choice' == driver.title, 'Returned product name is different - {}'.format(X)

html = driver.page_source
driver.quit()

In [308]:
html

'<html xmlns="http://www.w3.org/1999/xhtml" lang="en"><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta lang="en"><link rel="canonical" href="https://www.formbot3d.com/products/voron-24-corexy-3d-printer-kit-with-different-print-sizes-for-choice">\n<link rel="shortcut icon" href="/u_file/1612/photo/0548ba83b6.ico">\n<meta name="keywords" content="Voron 2.4 R2 Version CoreXY 3D Printer Kit,Voron">\n<meta name="description" content="Voron 2.4 R2 Version CoreXY 3D Printer Kit,Voron,3D Printer">\n<title>Voron 2.4 CoreXY 3D Printer Kit with Different Print Sizes for Choice</title><meta property="og:title" content="Voron 2.4 R2 Version CoreXY 3D Printer Kit"><meta property="og:type" content="product"><meta property="og:url" content="https://www.formbot3d.com/products/voron-24-corexy-3d-printer-kit-with-different-print-sizes-for-choice"><meta property="og:image" content="https://www.formbot3d.com/u_file/2101/products/23/58e29ae441.jpg"><meta property="og:descr

In [309]:
soup = bs(html)
soup

<html lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta lang="en"/><link href="https://www.formbot3d.com/products/voron-24-corexy-3d-printer-kit-with-different-print-sizes-for-choice" rel="canonical"/>
<link href="/u_file/1612/photo/0548ba83b6.ico" rel="shortcut icon"/>
<meta content="Voron 2.4 R2 Version CoreXY 3D Printer Kit,Voron" name="keywords"/>
<meta content="Voron 2.4 R2 Version CoreXY 3D Printer Kit,Voron,3D Printer" name="description"/>
<title>Voron 2.4 CoreXY 3D Printer Kit with Different Print Sizes for Choice</title><meta content="Voron 2.4 R2 Version CoreXY 3D Printer Kit" property="og:title"/><meta content="product" property="og:type"/><meta content="https://www.formbot3d.com/products/voron-24-corexy-3d-printer-kit-with-different-print-sizes-for-choice" property="og:url"/><meta content="https://www.formbot3d.com/u_file/2101/products/23/58e29ae441.jpg" property="og:image"/><meta content="Voron 

check every product's stock level

In [310]:
# check every product's stock level
def findAllNameIDpairs(s):
    dic = {}
    for i in s.find_all('div', class_=re.compile('btn_attr GoodBorderColor GoodBorderHoverColor')):
        dic[i.get('value')] = i.get('title')
    return dic

In [404]:
# check every product's stock level
def findAllStocks(s):
    for i in s.find_all('input', id=='ext_attr'):
        if re.search(r'\d{3}_\d{3}', i.get('value')):
            values = json.loads(i.get('value'))

    dic = findAllNameIDpairs(s)
    
    print_size = []
    hotend_type = []
    ships_from = []
    stocks = []
    for k, v in values.items():
        a, b, c = [dic[i] for i in k.split('_')]
        print_size.append(a)
        hotend_type.append(b)
        ships_from.append(c)
        stocks.append(v[1])
    
    return print_size, hotend_type, ships_from, stocks

In [405]:
print_size2, hotend_type2, ships_from2, stocks2 = findAllStocks(soup)

In [406]:
df2 = pd.DataFrame({'Print Size': print_size2, 
                    'Hotend Type': hotend_type2,
                    'Ships From': ships_from2,
                    'Stocks': stocks2})
df2.sort_values('Stocks', ascending=False).head()

,Print Size,Hotend Type,Ships From,Stocks
26,350*350*350mm,V6 Hotend,Czech,7
16,300*300*300mm,Dragon Standard Flow,China,63
12,300*300*300mm,V6 Hotend,China,62
28,350*350*350mm,Dragon Standard Flow,China,60
20,300*300*300mm,Dragon High Flow,China,58


check available product's stock level

In [314]:
print_sizes = list(set(print_size2))
hotend_types = list(set(hotend_type2))
ships_froms = list(set(ships_from2))

In [436]:
def findPrice(s):
    return s.find(class_='price themes_products_price').string[:-3]

In [437]:
print_size = []
hotend_type = []
ships_from = []
price = []

# initialize and close the driver (new): 
driver = webdriver.Chrome(path)

# initialize and close the driver (old): error SessionNotCreatedException
# driver = webdriver.Chrome(path)

# make sure that the page has time to properly open 
#time.sleep(3) 

driver.get(url)

# check if it is the right product
assert 'Voron 2.4 CoreXY 3D Printer Kit with Different Print Sizes for Choice' == driver.title, 'Returned product name is different - {}'.format(X)

# click on buttons to get the specific product
for p in print_sizes:
    driver.find_element_by_xpath("//div[@title='{}']".format(p)).click()
    for h in hotend_types:
        driver.find_element_by_xpath("//div[@title='{}']".format(h)).click()
        for s in ships_froms:
            print_size.append(p)
            hotend_type.append(h)
            ships_from.append(s)
            driver.find_element_by_xpath("//div[@title='{}']".format(s)).click()
            print('Scraping product -', (p, h, s))
            
            # scrape
            html = driver.page_source            
            price.append(findPrice(bs(html)))
            
            time.sleep(1)

driver.quit()


Scraping product - ('300*300*300mm', 'Dragon Standard Flow', 'Czech')
Scraping product - ('300*300*300mm', 'Dragon Standard Flow', 'United States')
Scraping product - ('300*300*300mm', 'Dragon Standard Flow', 'China')
Scraping product - ('300*300*300mm', 'Dragon Standard Flow', 'Australia')
Scraping product - ('300*300*300mm', 'Dragon High Flow', 'Czech')
Scraping product - ('300*300*300mm', 'Dragon High Flow', 'United States')
Scraping product - ('300*300*300mm', 'Dragon High Flow', 'China')
Scraping product - ('300*300*300mm', 'Dragon High Flow', 'Australia')
Scraping product - ('300*300*300mm', 'V6 Hotend', 'Czech')
Scraping product - ('300*300*300mm', 'V6 Hotend', 'United States')
Scraping product - ('300*300*300mm', 'V6 Hotend', 'China')
Scraping product - ('300*300*300mm', 'V6 Hotend', 'Australia')
Scraping product - ('250*250*250mm', 'Dragon Standard Flow', 'Czech')
Scraping product - ('250*250*250mm', 'Dragon Standard Flow', 'United States')
Scraping product - ('250*250*250mm',

In [438]:
df = pd.DataFrame({'Print Size': print_size, 
                   'Hotend Type': hotend_type,
                   'Ships From': ships_from,
                   'Price': price})
df.head()

,Print Size,Hotend Type,Ships From,Price
0,300*300*300mm,Dragon Standard Flow,Czech,$894
1,300*300*300mm,Dragon Standard Flow,United States,$894
2,300*300*300mm,Dragon Standard Flow,China,$794
3,300*300*300mm,Dragon Standard Flow,Australia,$794
4,300*300*300mm,Dragon High Flow,Czech,$899


In [439]:
final_df = pd.merge(left=df, right=df2, on = ['Print Size', 'Hotend Type', 'Ships From'])
final_df = final_df[final_df['Stocks']!='0'].sort_values(['Price', 'Stocks'], ascending=[True, False])
final_df.head(10)

,Print Size,Hotend Type,Ships From,Price,Stocks
22,250*250*250mm,V6 Hotend,China,$679,42
14,250*250*250mm,Dragon Standard Flow,China,$679,34
10,300*300*300mm,V6 Hotend,China,$739,62
18,250*250*250mm,Dragon High Flow,China,$739,39
2,300*300*300mm,Dragon Standard Flow,China,$794,63
6,300*300*300mm,Dragon High Flow,China,$799,58
34,350*350*350mm,V6 Hotend,China,$799,44
9,300*300*300mm,V6 Hotend,United States,$839,37
8,300*300*300mm,V6 Hotend,Czech,$839,21
26,350*350*350mm,Dragon Standard Flow,China,$854,60


In [440]:
# run the script manually to "save" the result to a file
with open("results.txt", "w") as f:
    for i in final_df.columns:
        f.write(str(i))
        f.write(',')
    f.write('\n')
    np.savetxt(f, final_df.values, delimiter=',', fmt='%s')            

In [441]:
# run the script with cron job to "print" the result to a file
print(','.join(list(final_df.columns)))
for i in final_df.values:
    print(','.join(list(i)))

Print Size,Hotend Type,Ships From,Price,Stocks
250*250*250mm,V6 Hotend,China,$679,42
250*250*250mm,Dragon Standard Flow,China,$679,34
300*300*300mm,V6 Hotend,China,$739,62
250*250*250mm,Dragon High Flow,China,$739,39
300*300*300mm,Dragon Standard Flow,China,$794,63
300*300*300mm,Dragon High Flow,China,$799,58
350*350*350mm,V6 Hotend,China,$799,44
300*300*300mm,V6 Hotend,United States,$839,37
300*300*300mm,V6 Hotend,Czech,$839,21
350*350*350mm,Dragon Standard Flow,China,$854,60
350*350*350mm,Dragon High Flow,China,$859,52
300*300*300mm,Dragon Standard Flow,United States,$894,42
300*300*300mm,Dragon Standard Flow,Czech,$894,24
350*350*350mm,V6 Hotend,Czech,$899,7
300*300*300mm,Dragon High Flow,United States,$899,48
350*350*350mm,V6 Hotend,United States,$899,42
300*300*300mm,Dragon High Flow,Czech,$899,31
350*350*350mm,Dragon Standard Flow,United States,$954,55
350*350*350mm,Dragon Standard Flow,Czech,$954,42
350*350*350mm,Dragon High Flow,United States,$959,35
350*350*350mm,Dragon High F